In [ ]:
#@title Install ComfyUI and Download Model
!apt -q install -y aria2
%cd /content/
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip -q install -r requirements.txt
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git || true
%cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
!aria2c -c -x16 -s16 -k1M https://huggingface.co/Comfy-Org/flux2-klein-4B/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors \
-d /content/ComfyUI/models/text_encoders -o qwen_3_4b.safetensors > /dev/null 2>&1 \
&& echo "✅ Text encoder download complete" || echo "❌ Text encoder download failed"


!aria2c -c -x16 -s16 -k1M https://huggingface.co/black-forest-labs/FLUX.2-klein-4b-fp8/resolve/main/flux-2-klein-4b-fp8.safetensors \
-d /content/ComfyUI/models/diffusion_models -o flux-2-klein-4b-fp8.safetensors > /dev/null 2>&1 \
&& echo "✅ Diffusion model download complete" || echo "❌ Diffusion model download failed"


!aria2c -c -x16 -s16 -k1M https://huggingface.co/Comfy-Org/flux2-dev/resolve/main/split_files/vae/flux2-vae.safetensors \
-d /content/ComfyUI/models/vae -o flux2-vae.safetensors > /dev/null 2>&1 \
&& echo "✅ VAE download complete" || echo "❌ VAE download failed"
import requests, json, os

url = "https://raw.githubusercontent.com/Comfy-Org/workflow_templates/refs/heads/main/templates/image_flux2_klein_image_edit_4b_distilled.json"

save_path = "/content/ComfyUI/user/default/workflows"
os.makedirs(save_path, exist_ok=True)

data = requests.get(url).json()

with open(f"{save_path}/image_flux2_klein_image_edit_4b_distilled.json", "w") as f:
    json.dump(data, f, indent=2)

print("✅ Workflow set as default")

from IPython.display import clear_output
clear_output()
from IPython.display import Audio,display
display(Audio("https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a", autoplay=True))

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 1s (1,995 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ..

In [ ]:

#@title Run Comfy UI
%cd /content
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || true

import subprocess, re,time ,socket,threading

def tunnel_printer(port):
    while True:
        time.sleep(0.5)
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            if s.connect_ex(("127.0.0.1", port)) == 0:
                break
        finally:
            s.close()
    print("\nComfyUI is up. Launching cloudflared tunnel...")
    p = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    for line in p.stderr:
        if "trycloudflare.com" in line:
            url = re.findall(r"https?://[^\s]+trycloudflare\.com[^\s]*", line)
            if url:
                print("\n🔗 Access ComfyUI:", url[0])

%cd /content/ComfyUI
PORT=8188
threading.Thread(target=tunnel_printer, args=(PORT,), daemon=True).start()

print("\nStarting ComfyUI...")
!python main.py --listen 0.0.0.0 --port {PORT} --dont-print-server
